# US Treasury Yield Analysis
This notebook compares different optimization engines (MOSEK, Gurobi, CPLEX) for minimizing funding costs in US Treasury bonds using real yield data from 2024.

In [1]:
library(tidyverse)
library(httr)

# Download and load the data
url <- "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_real_yield_curve&field_tdr_date_value=2024"
data <- content(GET(url), "text")
df <- read.csv(textConnection(data))

# Convert date to Date format
df$Date <- as.Date(df$Date)

# Plot the treasury yield curve
ggplot(df, aes(x=Date, y=Rate)) + 
  geom_line() + 
  labs(title="Treasury Yields 2024", x="Date", y="Yield")

In [2]:
# Calculate the relevant funding costs
df$FundingCost <- df$Rate * (1 + 0.01)  # Adding 1% transfer fee for illustration

# Plot Funding Costs
ggplot(df, aes(x=Date, y=FundingCost)) + 
  geom_line() + 
  labs(title="Funding Costs for US Treasury 2024", x="Date", y="Funding Cost")

In [3]:
# Optimization with MOSEK (R Interface to MOSEK required)
library(MOSEK)

optimize_mosek <- function(data) {
  # Define optimization variables and model using MOSEK API
  # For simplicity, assuming basic structure
  model <- mosek_model()  # Create a model
  model$objective <- sum(data$FundingCost * model$amounts)
  model$constraints <- c(sum(model$amounts) <= 100)  # Example constraint
  mosek_optimize(model)  # Optimize model
  return(model)
}

mo_model <- optimize_mosek(df)
print(paste('Optimal investment amounts using MOSEK:', mo_model$amounts))

In [4]:
# Optimization with Gurobi (R Interface to Gurobi required)
library(gurobi)

optimize_gurobi <- function(data) {
  # Define Gurobi model
  model <- list()
  model$obj <- data$FundingCost
  model$A <- matrix(1, nrow=1, ncol=12)
  model$sense <- "<="
  model$rhs <- 100
  result <- gurobi(model)
  return(result)
}

gurobi_result <- optimize_gurobi(df)
print(paste('Optimal investment amounts using Gurobi:', gurobi_result$x))

In [5]:
# Plot comparison
solvers <- c('MOSEK', 'Gurobi')
costs <- c(mo_model$cost, gurobi_result$objval)

barplot(costs, names.arg = solvers, main = "Comparison of Optimization Engines", ylab = "Minimized Funding Cost")